# Em's Debug File - currently working on concatenating ang and e_p onto latent z

In [2]:
import numpy as np
import tensorflow as tf
import h5py

In [6]:
def GetDataAngle(datafile, imgs3dscale =1, imgs3dpower=1, e_pscale = 100, angscale=1, angtype='theta', thresh=1e-4):
    print ('Loading Data from .....', datafile)
    f = h5py.File(datafile,'r')                    # load data into f variable
    ang = np.array(f.get(angtype))                 # ang is an array of angle data from f, one value is concatenated onto the latent vector
    imgs3d = np.array(f.get('ECAL'))* imgs3dscale    # imgs3d is a 3d array, cut from the cylinder that the calorimeter produces -has 25 layers along z-axis
    e_p = np.array(f.get('energy'))/e_pscale       # e_p is an array of scaled energy data from f, one value is concatenated onto the latent vector
    imgs3d[imgs3d < thresh] = 0        # when imgs3d values are less than the threshold, they are reset to 0
    
    # set imgs3d, e_p, and ang as float 32 datatypes
    imgs3d = imgs3d.astype(np.float32)
    e_p = e_p.astype(np.float32)
    ang = ang.astype(np.float32)
    
    imgs3d = np.expand_dims(imgs3d, axis=-1)         # insert a new axis at the beginning for imgs3d
    
    # sum along axis
    ecal = np.sum(imgs3d, axis=(1, 2, 3))    # summed imgs3d data, used for training the discriminator
     
    # imgs3d ^ imgs3dpower
    if imgs3dpower !=1.:
        imgs3d = np.power(imgs3d, imgs3dpower)
            
    # imgs3d=ecal data; e_p=energy data; ecal=summed imgs3d (used to train the discriminator); ang=angle data
    return imgs3d, e_p, ang, ecal

In [9]:
#load the data from a file using getdataangle()
imgs3d, e_p, ang, ecal = GetDataAngle('Ele_VarAngleMeas_100_200_005.h5')

Loading Data from ..... Ele_VarAngleMeas_100_200_005.h5


In [10]:
# uncomment to see examples of the values
#e_p[100]
#ang[100]

1.8596954

Strategy for concatenating a single energy value and angle value onto a 1d latent tensor of length 254

In [11]:
latent_dim = 256
z = tf.random.normal([latent_dim-2])
z = tf.concat([z, [e_p[0]], [ang[0]]], 0)
z

<tf.Tensor 'concat:0' shape=(256,) dtype=float32>

The variable fed into the original loss.py code: **tf.shape(real_image_input)[0]** is from sara_main.py and is just the batch size. 

In the code, I assign it: **z_batch_size = tf.shape(real_image_input)[0]**

for testing purposes in this file, I assign **z_batch_size** to a random int

Loss.py concatenation code -- subbed phase and batch_size variables

Adel will pass in ang and e_p as tensors with length = **z_batch_size = tf.shape(real_image_input)[0]**

In [28]:
# example function get the right size of vectors
phase = 4              # a random number I chose for testing
z_batch_size = 12      # a random number I chose for testing, in the code it will be tf.shape(real_image_input)[0]
start = (z_batch_size * (phase-1))
e_p = e_p[start:(start+z_batch_size)]   # select the batch size range desired
print("select batch size of energy values: ", e_p_vector.shape, ", type", type(e_p_vector))
ang = ang[start:(start+z_batch_size)]   # select the batch size range desired
print("select batch size of angle values: ", ang_vector.shape, ", type", type(ang_vector))

select batch size of energy values:  (12,) , type <class 'numpy.ndarray'>
select batch size of angle values:  (12,) , type <class 'numpy.ndarray'>


## The loss.py code -- takes in ang and e_p tensors or vectors of the correct batch size range
This is the code put into loss.py forward functions

In [31]:
#z_batch_size = tf.shape(real_image_input)[0]                 # this value should be an integer

e_p_tensor = tf.reshape(e_p_vector, [z_batch_size,1])   # need z_batch_size x 1
print("energy tensor: dimensions ", e_p_tensor.shape, ", type", type(e_p_tensor))
ang_tensor = tf.reshape(ang_vector, [z_batch_size,1])   # need z_batch_size x 1
print("angle tensor: dimensions ", ang_tensor.shape, ", type", type(ang_tensor))


z = tf.random.normal(shape=[z_batch_size, latent_dim-2])   
print("create a gaussian latent tensor: dimensions ", z.shape, ", type", type(z))
print('now, concatenate tensors of angle and energy values for each batch onto the latent vector')
z = tf.concat([z, e_p_tensor, ang_tensor], 1)    # shape = (z_batch_size, 256)
print("latent tensor: dimensions ", z.shape, ", type", type(z))

energy tensor: dimensions  (12, 1) , type <class 'tensorflow.python.framework.ops.Tensor'>
angle tensor: dimensions  (12, 1) , type <class 'tensorflow.python.framework.ops.Tensor'>
create a gaussian latent tensor: dimensions  (12, 254) , type <class 'tensorflow.python.framework.ops.Tensor'>
now, concatenate tensors of angle and energy values for each batch onto the latent vector
latent tensor: dimensions  (12, 256) , type <class 'tensorflow.python.framework.ops.Tensor'>
